# Documento Explicativo: Funcionamiento de Whisper en el Proyecto de Reconocimiento de Productos

## Introducción
Este proyecto utiliza la biblioteca **Whisper** de OpenAI para transcribir el audio de videos y detectar menciones de productos. Whisper es un modelo de inteligencia artificial basado en transformers que puede transcribir audio en múltiples idiomas con alta precisión. En este contexto, se integra en el servicio de video (`video_service.py`) para procesar archivos de video, extraer el audio y convertirlo en texto transcrito, que luego se anota con información de productos detectados visualmente.

## Cómo Funciona Whisper en el Proyecto
Whisper se utiliza en la función `process_video` del archivo `video_service.py`. Aquí se detalla el flujo:

1. **Carga del Modelo**: 
    - Whisper se carga de manera lazy usando la función `get_whisper_model()`. Esto asegura que el modelo (versión "base") se cargue solo una vez y se reutilice en llamadas posteriores, optimizando el rendimiento.
    - El modelo se configura para transcribir en español (`language="es"`).

2. **Transcripción del Audio**:
    - Una vez cargado el modelo, se llama a `model.transcribe(video_path, language="es")` para procesar el archivo de video.
    - Whisper extrae automáticamente el audio del video y genera una transcripción segmentada, incluyendo timestamps de inicio y fin para cada segmento de texto.
    - Los segmentos se almacenan en una lista de diccionarios con claves: `"start"`, `"end"` y `"text"`.

3. **Integración con Detección Visual**:
    - Paralelamente, el video se procesa para detectar productos usando SIFT (Scale-Invariant Feature Transform).
    - Las detecciones visuales se agregan y se usan para anotar el texto transcrito, agregando etiquetas como `(SKU:product_id)` donde se detectan productos en el audio o en momentos superpuestos.

4. **Anotación Final**:
    - El texto transcrito se anota con SKUs basados en la detección visual y, opcionalmente, con un CSV de productos usando `annotate_text_with_csv`.
    - Esto produce un texto final que combina la transcripción con información contextual de productos.

### Ventajas de Whisper en Este Proyecto
- **Precisión en Español**: Configurado para español, maneja acentos y jerga local.
- **Eficiencia**: Modelo "base" es ligero y rápido para prototipos.
- **Integración**: Funciona directamente con archivos de video sin necesidad de extraer audio manualmente.
- **Segmentación**: Proporciona timestamps, permitiendo sincronización con detecciones visuales.

### Limitaciones
- Requiere GPU para rendimiento óptimo; en CPU puede ser lento para videos largos.
- La transcripción puede tener errores en audio de baja calidad o con ruido de fondo.
- No maneja idiomas mixtos automáticamente (fijado a español).

## Pruebas Correspondientes
Para probar el funcionamiento de Whisper, utilizamos el archivo `video_service.py` y un video de prueba `video_test.mp4`. Asegúrate de tener instaladas las dependencias: `openai-whisper`, `opencv-python`, y otros listados en el proyecto.

### Código de Prueba
Ejecuta el siguiente código en un entorno con acceso a los archivos. Asume que tienes un `sift_engine` configurado (por ejemplo, una instancia de una clase SIFT con base de datos de productos).


In [ ]:
import sys
import os

# Asegurarnos de que el directorio raíz del proyecto esté en el path
sys.path.append(os.path.abspath('.'))

from services.video_service import process_video
from ml.models.sift_engine import get_sift_engine

# 1. Inicializar el motor SIFT (Carga la base de datos de productos)
try:
    sift_engine = get_sift_engine()
    print("Motor SIFT inicializado correctamente.")
except Exception as e:
    print(f"Error al inicializar SIFT: {e}")

### Ejecución del Procesamiento de Video
A continuación, procesamos el video `video_test.mp4`. La función `process_video` realizará los pasos mencionados: detección visual, agregación, transcripción con Whisper y anotación final.

In [ ]:
# 2. Definir la ruta del video de prueba
video_path = 'video_test.mp4'

if os.path.exists(video_path):
    print(f"Procesando video: {video_path}...")
    
    # Llamada principal al servicio
    # frame_every_seconds=1: Procesa un frame cada segundo para detección visual
    try:
        resultado = process_video(video_path, sift_engine, frame_every_seconds=1)
        
        print("\n--- Resultado Final ---")
        print(resultado)
    except Exception as e:
        print(f"Error durante el procesamiento: {e}")
else:
    print(f"El archivo {video_path} no existe. Por favor verifica la ruta.")

### Análisis de Resultados
El resultado de `process_video` es un string que contiene la transcripción del audio enriquecida.

- **Detección Visual**: Si el sistema detectó productos visualmente (usando SIFT) durante el video, esos productos se habrán agregado al contexto del texto.
- **Transcripción**: El texto base proviene de lo que Whisper escuchó en el video.
- **Anotación**: Si hubo coincidencias entre lo dicho y lo visto, o menciones directas detectadas vía CSV, verás etiquetas como `(SKU:...)` insertadas en el texto.

Este flujo permite que el chatbot tenga "ojos y oídos" sobre el contenido multimedia enviado por el usuario.